In [ ]:
import requests as r
import pandas as pd
import numpy as np
import time

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
timestr = time.strftime("%Y%m%d-%H%M%S")

In [ ]:
class HappyFarmer:
    def __init__(self):
        self.url = 'https://parseapi.back4app.com/'
        self.app_id = 'ASK_PTCAN'
        self.key = 'ASK_PTCAN'
        self.usr = 'ASK_PTCAN'
        self.pwd = 'ASK_PTCAN'

In [ ]:
hf = HappyFarmer()
headers = {
    'x-parse-application-id': hf.app_id,
    'x-parse-client-key': hf.key
}
def getToken():
    req = r.get(hf.url + 'login?username='+ hf.usr +'&password=' + hf.pwd + '&=', headers = headers)
    headers.update({'x-parse-session-token':req.json().get('sessionToken')})
    return headers

In [ ]:
getToken()

In [ ]:
class getData:
    def __init__(self, url):
        self.url = url
        self.data = r.get(hf.url + 'classes/' + url, headers = getToken()).json()
        res = self.data.get('results')
        self.tb = pd.DataFrame(res)

In [ ]:
class getID:
    def __init__(self, url):
        self.url = url
        self.data = r.get(hf.url + 'classes/' + url, headers = getToken()).json()
        self.name = self.data.get('Name')
        self.code = url.split('/')[1]+':'+self.name

In [ ]:
farmer = getData('Farmer?limit=1000')
farmer = farmer.tb

In [ ]:
farmer_dependencies = ['District',
                       'FarmerType',
                       'Gender',
                       'MaritalStatus',
                       'Province',
                       'SubDistrict',
                       'Village']

In [ ]:
farmer_dval = {}

In [ ]:
for fd in farmer_dependencies:
    fd_val = getData(fd+'?limit=1000')
    farmer_dval.update({fd:fd_val.data['results']})

In [ ]:
farmer_dval['Province']

In [ ]:
def generateID(param,val):
    df = pd.DataFrame(farmer_dval[param], columns=['Name','objectId'])
    df = df.loc[df['objectId'] == val]
    return val+':'+df['Name'].iloc[0]

In [ ]:
for fd in farmer_dependencies:
    farmer['_'+fd] = farmer[fd].fillna(0).apply(lambda x:np.nan if (x == 0) else generateID(fd, x['objectId']))

In [ ]:
farmer = farmer.drop(columns = farmer_dependencies)
farmer = farmer.drop(columns = ['MigrasiId','createdAt','updatedAt'])

In [ ]:
farmer['_Lat'] = farmer['Location'].fillna(0).apply(lambda x:np.nan if (x==0)  else x['latitude'])
farmer['_Long'] = farmer['Location'].fillna(0).apply(lambda x:np.nan if (x==0)  else x['longitude'])
farmer = farmer.drop(columns  = ['Location'])

In [ ]:
farmer['_Birthdate'] = farmer['BirthDate'].fillna(0).apply(lambda x:np.nan if (x==0) else x['iso'].split('T')[0])
farmer['BirthPlace'] = farmer['BirthPlace'].replace({'-': None})
farmer = farmer.drop(columns = ['BirthDate','CardPicture','Profile','BirthPlace','objectId'])

In [ ]:
#farmer.to_csv('./farmer'+timestr+'.csv')
#farm.to_csv('./farm'+timestr+'.csv')
#purchasing.to_csv('./purchasing'+timestr+'.csv')

In [ ]:
farmer = farmer.where((pd.notnull(farmer)), None)

In [ ]:
farmer = farmer.replace(r'\-', np.nan, regex=True)
farmer.replace('', np.nan, inplace=True)

In [ ]:
farmer['_Address'] = farmer['_Province']+'|'+farmer['_District']+'|'+farmer['_SubDistrict']+'|'+farmer['_Village']

In [ ]:
farmer_dc = pd.read_excel('data-cleaning/DATA_CLEANING-FARMER-v11.xlsx', skiprows=[0])
farmer_dc = farmer_dc.drop(farmer_dc.columns[farmer_dc.columns.str.contains('Unnamed',case = False)],axis = 1)

In [ ]:
farmer_dc_cols = {'Address':'12690131|Detail Address',
 'Email':'18050004|Email',
 'Name':'32040001|Name',
 'PhoneNumber':'16040001|Phone Number',
 'RegistrationNumber':'34740130|Registration Number',
 'Religion':'80003|Religion',
 '_Address':'20030004|Address',
 '_FarmerType':'8060002|Type of Farmer',
 '_Gender':'38040001|Gender',
 '_MaritalStatus':'18050002|Martial Status',
 '_Lat':'14040002|Latitude',
 '_Long':'--GEOLON--|Longitude',
 '_birthdate':'18050003|Date of Birth'}

In [ ]:
farmer = farmer.rename(columns=farmer_dc_cols)

In [ ]:
farmer_cols = [c for c in farmer.columns if '|' in c]

In [ ]:
def addColumn(fdc):
    farmer[fdc] = np.nan
def dropColumn(fdc):
    df = farmer.drop(columns=fdc)
    return df

In [ ]:
[addColumn(fdc) for fdc in list(farmer_dc) if fdc not in farmer_cols]
remove_cols = [x for x in list(farmer) if x not in list(farmer_dc)]

In [ ]:
farmer = farmer.drop(columns=remove_cols)

In [ ]:
farmer = farmer[list(farmer_dc)]

In [ ]:
farmer.to_excel('./farmer'+timestr+'.xlsx', index=None)